# Lab 3: Advanced Partitioning Strategies

## 🎯 **Learning Objectives:**
- Understand different Kafka partitioning strategies
- Implement custom partitioners for business logic
- Analyze partition distribution patterns
- Learn when to use each partitioning approach

## 📚 **Key Concepts:**
1. **Round Robin Partitioning**: Even distribution across partitions
2. **Hash Key Partitioning**: Consistent partitioning by key
3. **Custom Partitioning**: Business logic-driven partitioning
4. **Partition Analysis**: Monitoring and optimization

## 🏗️ **Architecture Overview:**
```
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│   Producer      │───▶│   Kafka Topic    │───▶│   Consumer      │
│                 │    │   (3 Partitions) │    │                 │
└─────────────────┘    └──────────────────┘    └─────────────────┘
         │                        │                        │
         ▼                        ▼                        ▼
┌─────────────────┐    ┌──────────────────┐    ┌─────────────────┐
│ Partitioning    │    │ Partition 0      │    │ Consumer Group  │
│ Strategies:     │    │ Partition 1      │    │ Load Balancing  │
│ • Round Robin   │    │ Partition 2      │    │                 │
│ • Hash Key      │    │                  │    │                 │
│ • Custom Logic  │    │                  │    │                 │
└─────────────────┘    └──────────────────┘    └─────────────────┘
```


In [1]:
# Install and Import Dependencies
import json
import random
import time
from datetime import datetime
from collections import defaultdict
from kafka import KafkaProducer, KafkaConsumer, KafkaAdminClient
from kafka.errors import KafkaError
from kafka.admin import ConfigResource, ConfigResourceType
import threading


In [2]:
# Kafka Configuration
KAFKA_BOOTSTRAP_SERVERS = 'localhost:9092'
TOPIC_NAME = 'stock-data-partitioned'

# Partitioning Strategies
PARTITIONING_STRATEGIES = {
    'round_robin': 'Even distribution across partitions',
    'hash_key': 'Consistent partitioning by key',
    'custom_symbol': 'Business logic-driven partitioning',
    'volume_based': 'Volume-based partitioning',
    'time_based': 'Time-based partitioning'
}

print("🔧 Kafka Configuration:")
print(f"   Bootstrap Servers: {KAFKA_BOOTSTRAP_SERVERS}")
print(f"   Topic: {TOPIC_NAME}")
print(f"   Available Strategies: {list(PARTITIONING_STRATEGIES.keys())}")


🔧 Kafka Configuration:
   Bootstrap Servers: localhost:9092
   Topic: stock-data-partitioned
   Available Strategies: ['round_robin', 'hash_key', 'custom_symbol', 'volume_based', 'time_based']


In [3]:
# Create Topic with 3 Partitions
def create_topic_if_not_exists(topic_name: str, num_partitions: int = 3):
    """Create topic if it doesn't exist"""
    try:
        admin_client = KafkaAdminClient(
            bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
            client_id='partitioning_lab_admin'
        )
        
        # Check if topic exists
        metadata = admin_client.describe_cluster()
        print(f"✅ Connected to Kafka cluster")
        
        # Try to get topic metadata
        try:
            topic_metadata = admin_client.describe_topics([topic_name])
            topic_info = topic_metadata[topic_name]
            existing_partitions = len(topic_info.partitions)
            
            if existing_partitions == num_partitions:
                print(f"✅ Topic '{topic_name}' already exists with {num_partitions} partitions")
            else:
                print(f"⚠️ Topic '{topic_name}' exists but has {existing_partitions} partitions, need {num_partitions}")
                print(f"🔄 Deleting and recreating topic...")
                
                # Delete existing topic
                admin_client.delete_topics([topic_name])
                time.sleep(2)  # Wait for deletion
                
                # Create new topic with correct partitions
                from kafka.admin import NewTopic
                topic = NewTopic(
                    name=topic_name,
                    num_partitions=num_partitions,
                    replication_factor=1
                )
                admin_client.create_topics([topic])
                print(f"✅ Topic '{topic_name}' recreated with {num_partitions} partitions")
                
        except Exception as e:
            print(f"📝 Creating topic '{topic_name}' with {num_partitions} partitions...")
            from kafka.admin import NewTopic
            topic = NewTopic(
                name=topic_name,
                num_partitions=num_partitions,
                replication_factor=1
            )
            admin_client.create_topics([topic])
            print(f"✅ Topic '{topic_name}' created successfully")
        
        admin_client.close()
        
    except Exception as e:
        print(f"❌ Error managing topic: {e}")
        print("💡 Make sure Kafka is running: docker-compose up -d")

# Create the topic
create_topic_if_not_exists(TOPIC_NAME, 3)


✅ Connected to Kafka cluster
📝 Creating topic 'stock-data-partitioned' with 3 partitions...
❌ Error managing topic: [Error 36] TopicAlreadyExistsError: Request 'CreateTopicsRequest_v3(create_topic_requests=[(topic='stock-data-partitioned', num_partitions=3, replication_factor=1, replica_assignment=[], configs=[])], timeout=30000, validate_only=False)' failed with response 'CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='stock-data-partitioned', error_code=36, error_message="Topic 'stock-data-partitioned' already exists.")])'.
💡 Make sure Kafka is running: docker-compose up -d


In [4]:
# Custom Partitioner Classes
class CustomPartitioner:
    """Base class for custom partitioners"""
    def __init__(self, num_partitions: int = 3):
        self.num_partitions = num_partitions
    
    def partition(self, key: str, value: dict) -> int:
        """Determine partition for given key and value"""
        raise NotImplementedError("Subclasses must implement partition method")

class SymbolBasedPartitioner(CustomPartitioner):
    """Partitioner that groups stocks by company type"""
    def __init__(self, num_partitions: int = 3):
        super().__init__(num_partitions)
        # Define stock categories
        self.categories = {
            'tech_giants': ['AAPL', 'GOOGL', 'MSFT', 'META', 'NVDA'],
            'retail_tech': ['AMZN', 'TSLA'],
            'entertainment_enterprise': ['NFLX', 'ADBE', 'CRM']
        }
    
    def partition(self, key: str, value: dict) -> int:
        """Partition based on stock symbol category"""
        for partition, symbols in enumerate(self.categories.values()):
            if key in symbols:
                return partition
        return 0  # Default to first partition

class VolumeBasedPartitioner(CustomPartitioner):
    """Partitioner that groups stocks by trading volume"""
    def __init__(self, num_partitions: int = 3):
        super().__init__(num_partitions)
        self.volume_thresholds = [500000, 800000]  # Low, Medium, High
    
    def partition(self, key: str, value: dict) -> int:
        """Partition based on trading volume"""
        volume = value.get('volume', 0)
        for i, threshold in enumerate(self.volume_thresholds):
            if volume < threshold:
                return i
        return self.num_partitions - 1  # High volume

class TimeBasedPartitioner(CustomPartitioner):
    """Partitioner that groups messages by trading hours"""
    def __init__(self, num_partitions: int = 3):
        super().__init__(num_partitions)
        self.time_windows = [
            (9, 12),   # Morning trading
            (12, 15),  # Midday trading
            (15, 18)   # Afternoon trading
        ]
    
    def partition(self, key: str, value: dict) -> int:
        """Partition based on trading time"""
        timestamp_str = value.get('timestamp', '')
        try:
            # Extract hour from timestamp
            hour = int(timestamp_str.split('T')[1].split(':')[0])
            for i, (start, end) in enumerate(self.time_windows):
                if start <= hour < end:
                    return i
        except:
            pass
        return 0  # Default to morning

print("📋 Available partitioners:")
print("   - SymbolBasedPartitioner: Groups stocks by company type")
print("   - VolumeBasedPartitioner: Groups by trading volume")
print("   - TimeBasedPartitioner: Groups by trading time")


📋 Available partitioners:
   - SymbolBasedPartitioner: Groups stocks by company type
   - VolumeBasedPartitioner: Groups by trading volume
   - TimeBasedPartitioner: Groups by trading time


In [5]:
# Stock Data Generator with Partitioning Support
class StockDataGenerator:
    def __init__(self, bootstrap_servers: str, topic: str):
        self.topic = topic
        self.symbols = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN', 'META', 'NVDA', 'NFLX', 'ADBE', 'CRM']
        self.base_prices = {
            'AAPL': 150.0, 'GOOGL': 2800.0, 'MSFT': 350.0, 'TSLA': 250.0, 'AMZN': 3200.0,
            'META': 300.0, 'NVDA': 450.0, 'NFLX': 400.0, 'ADBE': 500.0, 'CRM': 200.0
        }
        self.partitioner = None
        self.partition_counts = defaultdict(int)
    
    def set_partitioner(self, partitioner: CustomPartitioner):
        """Set custom partitioner"""
        self.partitioner = partitioner
        self.partition_counts.clear()
    
    def generate_ohlcv_data(self, symbol: str, base_price: float) -> dict:
        """Generate realistic OHLCV data"""
        price_change = random.uniform(-0.02, 0.02)
        new_price = base_price * (1 + price_change)
        
        open_price = round(new_price * random.uniform(0.998, 1.002), 2)
        close_price = round(new_price * random.uniform(0.998, 1.002), 2)
        high_price = round(max(open_price, close_price) * random.uniform(1.001, 1.005), 2)
        low_price = round(min(open_price, close_price) * random.uniform(0.995, 0.999), 2)
        
        volume = random.randint(100000, 1000000)
        
        return {
            "symbol": symbol,
            "timestamp": datetime.now().isoformat() + "Z",
            "open": open_price,
            "high": high_price,
            "low": low_price,
            "close": close_price,
            "volume": volume,
            "exchange": "NASDAQ"
        }
    
    def send_with_partitioning(self, num_messages: int = 20, strategy: str = 'round_robin'):
        """Send data using specified partitioning strategy"""
        print(f"📈 Generating {num_messages} messages with {strategy} partitioning...")
        
        # Create producer with appropriate configuration
        producer_config = {
            'bootstrap_servers': KAFKA_BOOTSTRAP_SERVERS,
            'value_serializer': lambda v: json.dumps(v).encode('utf-8'),
            'key_serializer': lambda k: k.encode('utf-8') if k else None
        }
        
        producer = KafkaProducer(**producer_config)
        
        for i in range(num_messages):
            symbol = random.choice(self.symbols)
            base_price = self.base_prices[symbol]
            ohlcv_data = self.generate_ohlcv_data(symbol, base_price)
            
            # Determine partition based on strategy
            partition = self._determine_partition(strategy, symbol, ohlcv_data)
            
            # Send message
            future = producer.send(
                self.topic, 
                key=symbol, 
                value=ohlcv_data,
                partition=partition
            )
            
            # Track partition distribution
            self.partition_counts[partition] += 1
            
            try:
                record_metadata = future.get(timeout=10)
                print(f"📊 Sent {symbol}: ${ohlcv_data['close']} -> Partition {record_metadata.partition}")
            except KafkaError as e:
                print(f"❌ Error sending message: {e}")
            
            time.sleep(0.1)  # Small delay between messages
        
        producer.flush()
        producer.close()
        
        print(f"✅ Successfully sent {num_messages} messages using {strategy} strategy")
        print(f"📊 Partition distribution: {dict(self.partition_counts)}")
    
    def _determine_partition(self, strategy: str, key: str, value: dict) -> int:
        """Determine partition based on strategy"""
        num_partitions = 3  # Default number of partitions
        
        if strategy == 'round_robin':
            # Round robin - distribute evenly
            return hash(key + str(time.time())) % num_partitions
        
        elif strategy == 'hash_key':
            # Hash key - same key goes to same partition
            return hash(key) % num_partitions
        
        elif strategy == 'custom_symbol' and self.partitioner:
            # Use custom symbol-based partitioner
            return self.partitioner.partition(key, value)
        
        elif strategy == 'volume_based' and self.partitioner:
            # Use volume-based partitioner
            return self.partitioner.partition(key, value)
        
        elif strategy == 'time_based' and self.partitioner:
            # Use time-based partitioner
            return self.partitioner.partition(key, value)
        
        else:
            # Default to round robin
            return hash(key + str(time.time())) % num_partitions

# Initialize data generator
data_generator = StockDataGenerator(KAFKA_BOOTSTRAP_SERVERS, TOPIC_NAME)


## Exercise 1: Round Robin Partitioning

### 🎯 **Learning Objectives:**
- Understand round robin partitioning
- Observe even distribution across partitions
- Learn when to use round robin strategy

### 📚 **Key Concepts:**
1. **Round Robin**: Messages distributed evenly across partitions
2. **Load Balancing**: Ensures equal load on all partitions
3. **Use Case**: When you want uniform distribution regardless of key


In [6]:
# Exercise 1: Round Robin Partitioning Demo
print("🔄 Demonstrating Round Robin Partitioning...")

# Send data with round robin strategy
data_generator.send_with_partitioning(num_messages=15, strategy='round_robin')

print("\n📊 Round Robin Analysis:")
print("✅ Round robin distributes messages evenly across partitions")
print("✅ No correlation between message content and partition")
print("✅ Good for load balancing when key doesn't matter")


🔄 Demonstrating Round Robin Partitioning...
📈 Generating 15 messages with round_robin partitioning...
📊 Sent META: $299.82 -> Partition 1
📊 Sent AAPL: $152.74 -> Partition 1
📊 Sent AMZN: $3238.51 -> Partition 0
📊 Sent NFLX: $402.28 -> Partition 1
📊 Sent CRM: $196.36 -> Partition 0
📊 Sent NFLX: $405.79 -> Partition 0
📊 Sent AAPL: $148.33 -> Partition 1
📊 Sent NVDA: $457.92 -> Partition 0
📊 Sent TSLA: $250.1 -> Partition 1
📊 Sent NFLX: $401.27 -> Partition 0
📊 Sent TSLA: $245.5 -> Partition 1
📊 Sent NFLX: $395.34 -> Partition 0
📊 Sent AAPL: $150.38 -> Partition 2
📊 Sent CRM: $200.99 -> Partition 0
📊 Sent GOOGL: $2763.24 -> Partition 2
✅ Successfully sent 15 messages using round_robin strategy
📊 Partition distribution: {1: 6, 0: 7, 2: 2}

📊 Round Robin Analysis:
✅ Round robin distributes messages evenly across partitions
✅ No correlation between message content and partition
✅ Good for load balancing when key doesn't matter


## Exercise 2: Hash Key Partitioning

### 🎯 **Learning Objectives:**
- Understand hash key partitioning
- Learn about consistent partitioning
- Observe key-based distribution

### 📚 **Key Concepts:**
1. **Hash Key**: Same key always goes to same partition
2. **Consistency**: Ensures related messages are co-located
3. **Use Case**: When you need to group related messages together


In [7]:
# Exercise 2: Hash Key Partitioning Demo
print("🔑 Demonstrating Hash Key Partitioning...")

# Send data with hash key strategy
data_generator.send_with_partitioning(num_messages=15, strategy='hash_key')

print("\n📊 Hash Key Analysis:")
print("✅ Same stock symbol always goes to same partition")
print("✅ Related messages are co-located")
print("✅ Good for maintaining order and grouping related data")
print("✅ Enables efficient processing by symbol")


🔑 Demonstrating Hash Key Partitioning...
📈 Generating 15 messages with hash_key partitioning...
📊 Sent NVDA: $450.63 -> Partition 2
📊 Sent GOOGL: $2810.52 -> Partition 0
📊 Sent META: $299.98 -> Partition 0
📊 Sent NFLX: $400.58 -> Partition 1
📊 Sent META: $297.14 -> Partition 0
📊 Sent NVDA: $446.63 -> Partition 2
📊 Sent NVDA: $448.79 -> Partition 2
📊 Sent NVDA: $448.89 -> Partition 2
📊 Sent TSLA: $254.94 -> Partition 2
📊 Sent AMZN: $3142.29 -> Partition 2
📊 Sent NFLX: $407.95 -> Partition 1
📊 Sent GOOGL: $2743.45 -> Partition 0
📊 Sent NFLX: $402.02 -> Partition 1
📊 Sent ADBE: $501.4 -> Partition 2
📊 Sent MSFT: $349.89 -> Partition 0
✅ Successfully sent 15 messages using hash_key strategy
📊 Partition distribution: {1: 9, 0: 12, 2: 9}

📊 Hash Key Analysis:
✅ Same stock symbol always goes to same partition
✅ Related messages are co-located
✅ Good for maintaining order and grouping related data
✅ Enables efficient processing by symbol


## Exercise 3: Custom Symbol-Based Partitioning

### 🎯 **Learning Objectives:**
- Implement custom partitioning logic
- Group stocks by business category
- Understand business-driven partitioning

### 📚 **Key Concepts:**
1. **Custom Logic**: Define your own partitioning rules
2. **Business Grouping**: Group related companies together
3. **Use Case**: When you need domain-specific partitioning


In [8]:
# Exercise 3: Custom Symbol-Based Partitioning Demo
print("🏢 Demonstrating Custom Symbol-Based Partitioning...")

# Set up custom symbol-based partitioner
symbol_partitioner = SymbolBasedPartitioner(num_partitions=3)
data_generator.set_partitioner(symbol_partitioner)

# Send data with custom symbol strategy
data_generator.send_with_partitioning(num_messages=20, strategy='custom_symbol')

print("\n📊 Custom Symbol Analysis:")
print("✅ Stocks grouped by business category:")
print("   - Partition 0: Tech Giants (AAPL, GOOGL, MSFT, META, NVDA)")
print("   - Partition 1: Retail Tech (AMZN, TSLA)")
print("   - Partition 2: Entertainment & Enterprise (NFLX, ADBE, CRM)")
print("✅ Enables category-specific processing")
print("✅ Good for sector-based analytics")


🏢 Demonstrating Custom Symbol-Based Partitioning...
📈 Generating 20 messages with custom_symbol partitioning...
📊 Sent CRM: $202.44 -> Partition 2
📊 Sent MSFT: $356.29 -> Partition 0
📊 Sent NVDA: $451.37 -> Partition 0
📊 Sent TSLA: $251.84 -> Partition 1
📊 Sent ADBE: $497.7 -> Partition 2
📊 Sent NVDA: $447.11 -> Partition 0
📊 Sent AAPL: $152.78 -> Partition 0
📊 Sent CRM: $199.92 -> Partition 2
📊 Sent AAPL: $148.71 -> Partition 0
📊 Sent ADBE: $500.06 -> Partition 2
📊 Sent AMZN: $3224.77 -> Partition 1
📊 Sent MSFT: $355.61 -> Partition 0
📊 Sent MSFT: $348.47 -> Partition 0
📊 Sent MSFT: $342.8 -> Partition 0
📊 Sent CRM: $197.79 -> Partition 2
📊 Sent GOOGL: $2810.64 -> Partition 0
📊 Sent GOOGL: $2757.68 -> Partition 0
📊 Sent MSFT: $346.89 -> Partition 0
📊 Sent NFLX: $393.41 -> Partition 2
📊 Sent ADBE: $502.24 -> Partition 2
✅ Successfully sent 20 messages using custom_symbol strategy
📊 Partition distribution: {2: 7, 0: 11, 1: 2}

📊 Custom Symbol Analysis:
✅ Stocks grouped by business categ

## Exercise 4: Volume-Based Partitioning

### 🎯 **Learning Objectives:**
- Implement volume-based partitioning
- Group stocks by trading volume
- Understand performance-driven partitioning

### 📚 **Key Concepts:**
1. **Volume Thresholds**: Define volume ranges for partitions
2. **Performance Optimization**: Group by processing requirements
3. **Use Case**: When processing complexity varies by volume


In [9]:
# Exercise 4: Volume-Based Partitioning Demo
print("📊 Demonstrating Volume-Based Partitioning...")

# Set up volume-based partitioner
volume_partitioner = VolumeBasedPartitioner(num_partitions=3)
data_generator.set_partitioner(volume_partitioner)

# Send data with volume-based strategy
data_generator.send_with_partitioning(num_messages=20, strategy='volume_based')

print("\n📊 Volume-Based Analysis:")
print("✅ Stocks grouped by trading volume:")
print("   - Partition 0: Low Volume (< 500K)")
print("   - Partition 1: Medium Volume (500K - 800K)")
print("   - Partition 2: High Volume (> 800K)")
print("✅ Enables volume-specific processing")
print("✅ Good for performance optimization")


📊 Demonstrating Volume-Based Partitioning...
📈 Generating 20 messages with volume_based partitioning...
📊 Sent GOOGL: $2767.06 -> Partition 2
📊 Sent CRM: $204.23 -> Partition 1
📊 Sent NVDA: $449.64 -> Partition 0
📊 Sent NFLX: $403.58 -> Partition 0
📊 Sent NVDA: $455.97 -> Partition 0
📊 Sent ADBE: $507.3 -> Partition 0
📊 Sent AMZN: $3142.3 -> Partition 1
📊 Sent NVDA: $440.5 -> Partition 0
📊 Sent AAPL: $147.59 -> Partition 0
📊 Sent CRM: $197.66 -> Partition 0
📊 Sent META: $304.51 -> Partition 0
📊 Sent AAPL: $151.85 -> Partition 1
📊 Sent AMZN: $3142.63 -> Partition 0
📊 Sent ADBE: $497.0 -> Partition 0
📊 Sent AMZN: $3239.03 -> Partition 1
📊 Sent META: $297.62 -> Partition 0
📊 Sent GOOGL: $2785.47 -> Partition 0
📊 Sent NVDA: $455.19 -> Partition 0
📊 Sent NFLX: $408.43 -> Partition 0
📊 Sent NVDA: $459.54 -> Partition 1
✅ Successfully sent 20 messages using volume_based strategy
📊 Partition distribution: {2: 1, 1: 5, 0: 14}

📊 Volume-Based Analysis:
✅ Stocks grouped by trading volume:
   - Pa

## Exercise 5: Time-Based Partitioning

### 🎯 **Learning Objectives:**
- Implement time-based partitioning
- Group messages by trading hours
- Understand temporal partitioning

### 📚 **Key Concepts:**
1. **Time Windows**: Partition by time periods
2. **Temporal Grouping**: Group by trading hours
3. **Use Case**: When processing varies by time


In [10]:
# Exercise 5: Time-Based Partitioning Demo
print("⏰ Demonstrating Time-Based Partitioning...")

# Set up time-based partitioner
time_partitioner = TimeBasedPartitioner(num_partitions=3)
data_generator.set_partitioner(time_partitioner)

# Send data with time-based strategy
data_generator.send_with_partitioning(num_messages=20, strategy='time_based')

print("\n📊 Time-Based Analysis:")
print("✅ Messages grouped by trading hours:")
print("   - Partition 0: Morning Trading (9:00 - 12:00)")
print("   - Partition 1: Midday Trading (12:00 - 15:00)")
print("   - Partition 2: Afternoon Trading (15:00+)")
print("✅ Enables time-specific processing")
print("✅ Good for temporal analytics")


⏰ Demonstrating Time-Based Partitioning...
📈 Generating 20 messages with time_based partitioning...
📊 Sent META: $297.47 -> Partition 0
📊 Sent AMZN: $3211.35 -> Partition 0
📊 Sent GOOGL: $2834.65 -> Partition 0
📊 Sent ADBE: $508.34 -> Partition 0
📊 Sent TSLA: $246.06 -> Partition 0
📊 Sent ADBE: $508.32 -> Partition 0
📊 Sent CRM: $203.59 -> Partition 0
📊 Sent NVDA: $445.35 -> Partition 0
📊 Sent NVDA: $449.38 -> Partition 0
📊 Sent NVDA: $446.24 -> Partition 0
📊 Sent MSFT: $349.78 -> Partition 0
📊 Sent CRM: $200.31 -> Partition 0
📊 Sent AMZN: $3172.99 -> Partition 0
📊 Sent NFLX: $408.4 -> Partition 0
📊 Sent META: $301.34 -> Partition 0
📊 Sent MSFT: $353.28 -> Partition 0
📊 Sent NVDA: $443.22 -> Partition 0
📊 Sent META: $303.69 -> Partition 0
📊 Sent MSFT: $357.42 -> Partition 0
📊 Sent NVDA: $446.16 -> Partition 0
✅ Successfully sent 20 messages using time_based strategy
📊 Partition distribution: {0: 20}

📊 Time-Based Analysis:
✅ Messages grouped by trading hours:
   - Partition 0: Morning 

## Exercise 6: Partition Analysis and Visualization

### 🎯 **Learning Objectives:**
- Analyze partition distribution patterns
- Visualize partitioning strategies
- Compare different approaches

### 📚 **Key Concepts:**
1. **Distribution Analysis**: Measure partition balance
2. **Visualization**: Create charts to understand patterns
3. **Comparison**: Compare different strategies


In [11]:
# Exercise 6: Partition Analysis and Visualization
print("📊 Analyzing Partition Distribution Patterns...")

def analyze_partition_distribution(strategy_name: str, partition_counts: dict):
    """Analyze and display partition distribution"""
    total_messages = sum(partition_counts.values())
    print(f"\n📈 {strategy_name} Distribution Analysis:")
    
    for partition, count in sorted(partition_counts.items()):
        percentage = (count / total_messages) * 100 if total_messages > 0 else 0
        bar = "█" * int(percentage / 5)  # Scale bar to fit
        print(f"   Partition {partition}: {count:3d} messages ({percentage:5.1f}%) {bar}")
    
    # Calculate balance metrics
    if partition_counts:
        counts = list(partition_counts.values())
        min_count = min(counts)
        max_count = max(counts)
        balance_ratio = min_count / max_count if max_count > 0 else 1.0
        
        print(f"   Balance Ratio: {balance_ratio:.3f} (1.0 = perfect balance)")
        print(f"   Min-Max Range: {max_count - min_count} messages")

# Analyze all strategies
strategies = ['round_robin', 'hash_key', 'custom_symbol', 'volume_based', 'time_based']
strategy_results = {}

for strategy in strategies:
    print(f"\n🔄 Testing {strategy} strategy...")
    
    # Reset partitioner based on strategy
    if strategy == 'custom_symbol':
        data_generator.set_partitioner(SymbolBasedPartitioner(3))
    elif strategy == 'volume_based':
        data_generator.set_partitioner(VolumeBasedPartitioner(3))
    elif strategy == 'time_based':
        data_generator.set_partitioner(TimeBasedPartitioner(3))
    else:
        data_generator.set_partitioner(None)
    
    # Send test data
    data_generator.send_with_partitioning(num_messages=30, strategy=strategy)
    
    # Store results
    strategy_results[strategy] = dict(data_generator.partition_counts)
    
    # Analyze distribution
    analyze_partition_distribution(strategy, data_generator.partition_counts)

print("\n✅ Partition analysis completed!")


📊 Analyzing Partition Distribution Patterns...

🔄 Testing round_robin strategy...
📈 Generating 30 messages with round_robin partitioning...
📊 Sent NVDA: $457.77 -> Partition 2
📊 Sent AMZN: $3192.8 -> Partition 1
📊 Sent AAPL: $147.13 -> Partition 2
📊 Sent META: $296.64 -> Partition 1
📊 Sent AMZN: $3138.31 -> Partition 2
📊 Sent ADBE: $503.47 -> Partition 2
📊 Sent MSFT: $354.92 -> Partition 0
📊 Sent CRM: $198.53 -> Partition 2
📊 Sent AAPL: $151.64 -> Partition 2
📊 Sent AAPL: $149.88 -> Partition 0
📊 Sent AMZN: $3217.4 -> Partition 1
📊 Sent AAPL: $152.36 -> Partition 0
📊 Sent AMZN: $3175.4 -> Partition 0
📊 Sent GOOGL: $2751.72 -> Partition 1
📊 Sent MSFT: $344.42 -> Partition 1
📊 Sent GOOGL: $2788.53 -> Partition 2
📊 Sent TSLA: $249.66 -> Partition 0
📊 Sent NVDA: $448.44 -> Partition 0
📊 Sent AMZN: $3169.09 -> Partition 1
📊 Sent META: $298.67 -> Partition 1
📊 Sent NVDA: $449.86 -> Partition 2
📊 Sent META: $295.76 -> Partition 2
📊 Sent CRM: $197.59 -> Partition 0
📊 Sent META: $298.28 -> Part

## Exercise 7: Best Practices and Cleanup

### 🎯 **Learning Objectives:**
- Learn partitioning best practices
- Understand when to use each strategy
- Review key takeaways

### 📚 **Best Practices:**
1. **Choose the right strategy** for your use case
2. **Monitor partition balance** to avoid hotspots
3. **Consider data locality** for performance
4. **Plan for rebalancing** when adding partitions


In [12]:
# Exercise 7: Best Practices and Cleanup
print("📚 Partitioning Strategy Best Practices...")

print("\n🎯 When to Use Each Strategy:")
print("\n1. 🔄 Round Robin:")
print("   ✅ Use when: Load balancing is priority")
print("   ✅ Use when: Key doesn't matter for processing")
print("   ❌ Avoid when: You need related messages together")

print("\n2. 🔑 Hash Key:")
print("   ✅ Use when: Same key should go to same partition")
print("   ✅ Use when: Maintaining order is important")
print("   ✅ Use when: Related messages need to be co-located")
print("   ❌ Avoid when: Keys are not evenly distributed")

print("\n3. 🏢 Custom Symbol-Based:")
print("   ✅ Use when: Business logic drives partitioning")
print("   ✅ Use when: Domain-specific grouping is needed")
print("   ✅ Use when: Different processing per category")
print("   ❌ Avoid when: Categories are not well-defined")

print("\n4. 📊 Volume-Based:")
print("   ✅ Use when: Processing complexity varies by volume")
print("   ✅ Use when: Performance optimization is critical")
print("   ✅ Use when: Different resources per volume tier")
print("   ❌ Avoid when: Volume distribution is unpredictable")

print("\n5. ⏰ Time-Based:")
print("   ✅ Use when: Processing varies by time")
print("   ✅ Use when: Temporal analytics are important")
print("   ✅ Use when: Different processing per time window")
print("   ❌ Avoid when: Time patterns are not meaningful")

print("\n📋 General Best Practices:")
print("✅ Monitor partition balance regularly")
print("✅ Avoid hotspots (uneven distribution)")
print("✅ Consider data locality for performance")
print("✅ Plan for partition rebalancing")
print("✅ Test partitioning strategies with real data")
print("✅ Document partitioning logic clearly")

print("\n🎯 Key Takeaways:")
print("1. Partitioning strategy affects performance and processing")
print("2. Choose strategy based on your specific use case")
print("3. Monitor and adjust partitioning as needed")
print("4. Consider both current and future requirements")
print("5. Test thoroughly with realistic data volumes")

print("\n✅ Lab 3: Advanced Partitioning Strategies completed!")
print("🚀 Next: Try Lab 4: Offset Management or Lab 5: Real-time Analytics")


📚 Partitioning Strategy Best Practices...

🎯 When to Use Each Strategy:

1. 🔄 Round Robin:
   ✅ Use when: Load balancing is priority
   ✅ Use when: Key doesn't matter for processing
   ❌ Avoid when: You need related messages together

2. 🔑 Hash Key:
   ✅ Use when: Same key should go to same partition
   ✅ Use when: Maintaining order is important
   ✅ Use when: Related messages need to be co-located
   ❌ Avoid when: Keys are not evenly distributed

3. 🏢 Custom Symbol-Based:
   ✅ Use when: Business logic drives partitioning
   ✅ Use when: Domain-specific grouping is needed
   ✅ Use when: Different processing per category
   ❌ Avoid when: Categories are not well-defined

4. 📊 Volume-Based:
   ✅ Use when: Processing complexity varies by volume
   ✅ Use when: Performance optimization is critical
   ✅ Use when: Different resources per volume tier
   ❌ Avoid when: Volume distribution is unpredictable

5. ⏰ Time-Based:
   ✅ Use when: Processing varies by time
   ✅ Use when: Temporal analytics 